In [2]:
import torch
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

c:\Users\kenne\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu


In [30]:
from torch import nn
from transformers import DistilBertModel

class DistilBERTLSTM(nn.Module):
    def __init__(self, hidden_size, num_layers):
        super(DistilBERTLSTM, self).__init__()
        self.distil_bert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        for param in self.distil_bert.parameters():
            param.requires_grad = False  # Freeze the BERT parameters
        self.lstm = nn.LSTM(input_size=768, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            outputs = self.distil_bert(input_ids, attention_mask)
            last_hidden_state = outputs.last_hidden_state()
        lstm_output, _ = self.lstm(last_hidden_state)
        lstm_output = lstm_output[:, -1, :]
        logits = self.fc(lstm_output)
        return logits


In [32]:
from torch.utils.data import DataLoader

class tokenize():
  def __init__(self, X):
    self.X = X
    self.tokenizer = tokenizer
  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    text = self.X

    encoded_input = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        padding='max_length',
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )

    input_ids = torch.tensor(encoded_input["input_ids"])
    attention_mask = encoded_input["attention_mask"]
    return input_ids.to(device), attention_mask.to(device)


In [33]:
model = DistilBERTLSTM(50, 2).to(device)
model.load_state_dict(torch.load("./model.pth", map_location="cpu"))
model.eval()


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DistilBERTLSTM(
  (distil_bert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): L

In [34]:
def classify(text):
  data = DataLoader(tokenize(text))
  with torch.inference_mode():
    num = 0
    denom = 0
    for inp_id, att_mask in data:
        logits = model(inp_id.squeeze(), att_mask).squeeze()
        preds = torch.round(torch.sigmoid(logits))
        num += preds.item()
        denom += 1
    return(num/denom)
classify(input("Input here: "))

C:\Users\kenne\AppData\Local\Temp\ipykernel_11020\310123347.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(encoded_input["input_ids"])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [28]:
import pandas as pd
import ast

df = pd.read_csv("final_data.csv")
for i in range(1):
    sent_vals = []
    comment_ls = ast.literal_eval(df.iloc[i,7])
    for comment in comment_ls:
        print(type(str(comment)))
        sent_vals.append(classify(comment))
    sent_avg = sum(sent_vals)/len(sent_vals)
    df.iloc[i,7] = sent_avg
    print(sent_vals, sent_avg)

<class 'str'>


C:\Users\kenne\AppData\Local\Temp\ipykernel_11020\310123347.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(encoded_input["input_ids"])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)